# 07 - Advanced Preprocessing with FSL: Eddy & TOPUP

This notebook provides a conceptual overview of advanced dMRI preprocessing steps that involve FSL (FMRIB Software Library) tools, specifically `topup` for susceptibility distortion correction and `eddy` for correcting eddy current distortions and subject motion.

The `diffusemri` library offers Python wrappers to interface with these powerful command-line tools, simplifying their integration into Python-based workflows. However, due to the external dependency on FSL and the complexity of input data required, this notebook will primarily focus on explaining the concepts and showing **conceptual examples** of how to use these wrappers and their corresponding CLI commands. **Actual execution of these FSL tools is not performed within this notebook.**

## <font color='red'>CRITICAL: FSL Dependency and Usage Note</font>

*   **FSL Installation Required**: To use the functionalities described in this notebook, you **must have FSL installed independently** on your system. The `diffusemri` library only provides wrappers; it does not install FSL.
*   **FSL in System PATH**: Ensure that the FSL tools (like `topup`, `applytopup`, `eddy_openmp`, or `eddy_cuda`) are accessible from your system's command-line PATH. If they are not, the Python wrappers will fail to execute them.
*   **Conceptual Examples**: The Python and CLI examples provided below use **placeholder file paths** (e.g., `/path/to/your/...`). You will need to replace these with paths to your actual data files. Setting up the correct input files (especially for `topup`, which requires specific b0 image pairs and acquisition parameter files) is crucial and non-trivial.
*   **Execution Time & Resources**: Running `eddy` and `topup` can be computationally intensive and time-consuming, especially `eddy` on large datasets. They may also require significant CPU, memory, and potentially GPU resources (for `eddy_cuda`).
*   **FSL Documentation**: For detailed information on FSL, its tools, installation, and the specific requirements for input files, please refer to the official [FSL Wiki](https://fsl.fmrib.ox.ac.uk/fsl/fslwiki/) and tool documentation (e.g., `eddy` user guide).

**This notebook aims to show *how* you would use the `diffusemri` wrappers, not to run a full FSL preprocessing pipeline here.**

In [ ]:
import os

# diffusemri library imports for FSL wrappers
from preprocessing.correction import correct_susceptibility_topup_fsl, correct_motion_eddy_fsl, load_eddy_outlier_report

print("diffusemri FSL wrapper functions imported conceptually.")
print("Remember: FSL must be installed and in your PATH for these to work in a real script.")

## Part 1: Susceptibility Distortion Correction with `topup`

Susceptibility-induced distortions are geometric distortions in MRI images caused by variations in magnetic susceptibility at air-tissue interfaces (e.g., near sinuses). These are particularly problematic in EPI sequences used for dMRI. FSL's `topup` tool estimates the susceptibility-induced off-resonance field and `applytopup` uses this field to correct images.

### Explanation of `topup`

*   **Purpose**: Estimates and corrects for susceptibility-induced off-resonance fields.
*   **Typical Inputs**:
    1.  **`imain_file`**: A NIfTI file containing a series of b0 images acquired with varying phase-encoding (PE) directions. Most commonly, this includes at least one pair of b0s with opposite PE directions (e.g., Anterior-Posterior (AP) and Posterior-Anterior (PA)).
    2.  **`encoding_file` (or `acqparams.txt`)**: A text file detailing the acquisition parameters for each volume in `imain_file`. Each row corresponds to a volume and typically contains:
        *   Phase-encoding vector (e.g., `0 1 0` for AP, `0 -1 0` for PA).
        *   Total EPI readout time (or effective echo spacing multiplied by EPI factor minus 1).
        *   Example for two volumes (AP then PA):
          ```
          0 1 0 0.065
          0 -1 0 0.065
          ```
    3.  **`config_file` (optional)**: FSL configuration file for `b02b0.cnf` is often used.
*   **Outputs**: `topup` generates a field coefficient file and a movement parameters file. `applytopup` then uses these to correct the input images or other images (like the full DWI series).

### Python Wrapper: `correct_susceptibility_topup_fsl`

The `diffusemri` library provides `correct_susceptibility_topup_fsl` to run `topup` and `applytopup`. 

**Function Signature (simplified):**
```python
def correct_susceptibility_topup_fsl(
    imain_file: str,                  # Path to merged AP/PA (or other PE) b0s
    encoding_file: str,               # Path to the acquisition parameters text file
    out_base_name: str,               # Basename for output files (e.g., 'topup_results')
    images_to_correct_file: str,      # Path to images that need correction using the field (e.g., full DWI or just the b0s)
    images_to_correct_encoding_indices: list[int], # List of row indices from encoding_file for images_to_correct_file
    output_type: str = "NIFTI_GZ",
    config_file: str = "b02b0.cnf",   # FSL config file for topup
    topup_kwargs: Optional[Dict[str, Any]] = None, # Extra args for topup
    applytopup_kwargs: Optional[Dict[str, Any]] = None # Extra args for applytopup
) -> Tuple[str, str, str, str] # (corrected_images_path, field_map_path, field_coeffs_path, movpar_path)
```

In [ ]:
print("--- Conceptual Python call for TOPUP ---")

# Define placeholder paths (REPLACE THESE WITH YOUR ACTUAL FILE PATHS)
conceptual_imain_AP_PA_b0s = "/path/to/your/merged_AP_PA_b0s.nii.gz" # e.g., 2 volumes: one AP b0, one PA b0
conceptual_encoding_file_txt = "/path/to/your/topup_acqparams.txt"
conceptual_output_base_for_topup = "/path/to/your_output_dir/topup_results"
# The images_to_correct_file would typically be your full DWI scan or the b0s themselves if that's all you have/need.
conceptual_images_to_correct_nifti = conceptual_imain_AP_PA_b0s # Example: correcting the b0s themselves
# If conceptual_images_to_correct_nifti has 2 volumes, and they correspond to the 2 lines in encoding_file:
conceptual_images_to_correct_indices = [0, 1] 

print(f"Conceptual call to correct_susceptibility_topup_fsl with placeholders:")
print(f"  imain_file='{conceptual_imain_AP_PA_b0s}'")
print(f"  encoding_file='{conceptual_encoding_file_txt}'")
print(f"  out_base_name='{conceptual_output_base_for_topup}'")
print(f"  images_to_correct_file='{conceptual_images_to_correct_nifti}'")
print(f"  images_to_correct_encoding_indices={conceptual_images_to_correct_indices}")

print("\n# --- Example content for conceptual_encoding_file_txt ---")
print("# 0 1 0 0.065   # Example for first volume (e.g., AP)")
print("# 0 -1 0 0.065  # Example for second volume (e.g., PA)")
print("# The last number is total readout time for that volume's acquisition.")

print("\n# --- Conceptual execution block (commented out) ---")
print("# try:")
print("#     corrected_images_path, field_map_path, field_coeffs_path, movpar_path = correct_susceptibility_topup_fsl(")
print("#         imain_file=conceptual_imain_AP_PA_b0s,")
print("#         encoding_file=conceptual_encoding_file_txt,")
print("#         out_base_name=conceptual_output_base_for_topup,")
print("#         images_to_correct_file=conceptual_images_to_correct_nifti,")
print("#         images_to_correct_encoding_indices=conceptual_images_to_correct_indices")
print("#     )")
print("#     print(f'  TOPUP completed. Corrected images: {corrected_images_path}')")
print("#     print(f'  Field map: {field_map_path}')")
print("#     print(f'  Field coefficients: {field_coeffs_path}')")
print("#     print(f'  Movement parameters: {movpar_path}')")
print("# except FileNotFoundError as fnf_error:")
print("#     print(f'  FSL tool not found. Is FSL installed and in PATH? Error: {fnf_error}')")
print("# except Exception as e:")
print("#     print(f'  Error during conceptual TOPUP call: {e}')")

print("\n# Note: This is a conceptual Python call. Actual execution requires a working FSL installation and valid input files.")

### CLI Usage: `run_preprocessing topup_fsl`

You can also perform `topup` correction using the `run_preprocessing.py` script:

```bash
# Conceptual CLI command - replace paths and ensure files are correctly formatted
python cli/run_preprocessing.py topup_fsl \
  --imain_file /path/to/your/merged_AP_PA_b0s.nii.gz \
  --encoding_file /path/to/your/topup_acqparams.txt \
  --images_to_correct_file /path/to/your/full_dwi_or_b0s.nii.gz \
  --images_to_correct_encoding_indices 0 1 # Example if correcting 2 volumes using first two acqparam lines
  --out_base_name /path/to/your_output_dir/topup_results
```

## Part 2: Motion and Eddy Current Correction with `eddy`

Eddy currents, induced by the strong diffusion gradients, and subject motion during the scan can cause significant artifacts and misalignments in dMRI data. FSL's `eddy` tool (often `eddy_openmp` for CPU or `eddy_cuda` for GPU versions) is designed to correct these distortions simultaneously. It can also use the field map information from `topup` for even more accurate correction (this is known as EPI distortion correction).

### Explanation of `eddy`

*   **Purpose**: Corrects for eddy current distortions, subject motion, and optionally EPI susceptibility distortions if `topup` results are provided.
*   **Typical Inputs**:
    1.  **`dwi_file`**: The main 4D dMRI NIfTI file.
    2.  **`mask_file`**: A binary brain mask NIfTI file for the dMRI data.
    3.  **`bval_file`**: Text file containing b-values.
    4.  **`bvec_file`**: Text file containing b-vectors.
    5.  **`index_file`**: A text file that maps each volume in the DWI series to a corresponding line in the `acqparams.txt` file. If all DWI volumes were acquired with the same parameters (e.g., same PE direction and readout time), this file will contain all 1s.
    6.  **`acqp_file` (`acqparams.txt`)**: Acquisition parameters file, similar to the one used by `topup`. It describes the PE direction and readout time for the acquisition group(s) defined in `index_file`.
    7.  **Optional `topup` outputs**: If `topup` was run, its `_fieldcoef.nii.gz` (or `_field.nii.gz`) and `_movpar.txt` files can be passed to `eddy` to aid in correcting susceptibility distortions.
*   **Outputs**: `eddy` produces a corrected 4D dMRI image and a text file with rotated b-vectors. It can also produce other outputs like outlier reports, CNR maps, etc.

### Python Wrapper: `correct_motion_eddy_fsl`

The `diffusemri` library's `correct_motion_eddy_fsl` function wraps the `eddy` tool.

**Function Signature (simplified):**
```python
def correct_motion_eddy_fsl(
    dwi_file: str, 
    bval_file: str, 
    bvec_file: str, 
    mask_file: str, 
    index_file: str, 
    acqp_file: str, 
    out_base_name: str, # Basename for output files
    use_cuda: bool = False, # If True, tries to use 'eddy_cuda'
    output_type: str = "NIFTI_GZ",
    **eddy_kwargs: Any # Additional arguments to pass to eddy
) -> Tuple[str, str, Optional[str], Optional[str]] # (corrected_dwi_path, rotated_bvecs_path, outlier_report_path, cnr_maps_path)
```
The `load_eddy_outlier_report(outlier_report_file: str)` function can be used to parse the outlier report generated by `eddy` (if `repol=True` is used).

In [ ]:
print("--- Conceptual Python call for Eddy ---")

# Define placeholder paths (REPLACE THESE WITH YOUR ACTUAL FILE PATHS)
conceptual_dwi_file = "/path/to/your/dwi_to_correct.nii.gz"
conceptual_mask_file = "/path/to/your/brain_mask.nii.gz"
conceptual_bvals_file = "/path/to/your/bvals.bval"
conceptual_bvecs_file = "/path/to/your/bvecs.bvec"
conceptual_index_file_for_eddy = "/path/to/your/eddy_index.txt"
conceptual_acqparams_file_for_eddy = "/path/to/your/eddy_acqparams.txt" # Often same as topup's
conceptual_output_base_for_eddy = "/path/to/your_output_dir/eddy_corrected_data"

# Optional: If you ran topup and want to use its results
conceptual_topup_fieldcoef = "/path/to/your_output_dir/topup_results_fieldcoef.nii.gz" # From topup output
conceptual_topup_movpar = "/path/to/your_output_dir/topup_results_movpar.txt"       # From topup output

print(f"Conceptual call to correct_motion_eddy_fsl with placeholders:")
print(f"  dwi_file='{conceptual_dwi_file}'")
print(f"  mask_file='{conceptual_mask_file}'")
print(f"  bval_file='{conceptual_bvals_file}'")
print(f"  bvec_file='{conceptual_bvecs_file}'")
print(f"  index_file='{conceptual_index_file_for_eddy}'")
print(f"  acqp_file='{conceptual_acqparams_file_for_eddy}'")
print(f"  out_base_name='{conceptual_output_base_for_eddy}'")

print("\n# --- Example content for conceptual_index_file_for_eddy ---")
print("# If all DWI volumes use the first (and perhaps only) line of acqparams_file:")
print("# 1 1 1 1 ... (one '1' for each DWI volume)")

print("\n# --- Conceptual execution block (commented out) ---")
print("# eddy_extra_args = {")
print("#     'repol': True,  # Example: Replace outliers detected by eddy")
print("#     'cnr_maps': True, # Example: Output CNR maps")
print("#     # If using TOPUP results for EPI distortion correction:
print("#     # 'in_topup_fieldcoef': conceptual_topup_fieldcoef, # or 'in_topup_field' if using the field directly
print("#     # 'in_topup_movpar': conceptual_topup_movpar,      
print("#     # 'in_topup_fwhm': 8, # Example FWHM for field
print("#     # 'in_topup_reg_model': 'matrix', # Example registration model
print("#     # 'in_topup_apply': True # Tell eddy to apply topup internally 
print("# }")
print("# try:")
print("#     corrected_dwi_path, rotated_bvecs_path, outlier_report_path, cnr_maps_path = correct_motion_eddy_fsl(")
print("#         dwi_file=conceptual_dwi_file,")
print("#         bval_file=conceptual_bvals_file,")
print("#         bvec_file=conceptual_bvecs_file,")
print("#         mask_file=conceptual_mask_file,")
print("#         index_file=conceptual_index_file_for_eddy,")
print("#         acqp_file=conceptual_acqparams_file_for_eddy,")
print("#         out_base_name=conceptual_output_base_for_eddy,")
print("#         use_cuda=False,  # Set to True if you have eddy_cuda and want to use it
print("#         **eddy_extra_args")
print("#     )")
print("#     print(f'  Eddy correction completed. Corrected DWI: {corrected_dwi_path}')")
print("#     print(f'  Rotated b-vectors: {rotated_bvecs_path}')")
print("#     if outlier_report_path:")
print("#         print(f'  Outlier report: {outlier_report_path}')")
print("#         outlier_summary = load_eddy_outlier_report(outlier_report_path)")
print("#         print(f'  Outlier summary: {outlier_summary}')")
print("#     if cnr_maps_path:")
print("#         print(f'  CNR maps: {cnr_maps_path}')")
print("# except FileNotFoundError as fnf_error:")
print("#     print(f'  FSL tool (e.g., eddy_openmp or eddy_cuda) not found. Is FSL installed and in PATH? Error: {fnf_error}')")
print("# except Exception as e:")
print("#     print(f'  Error during conceptual Eddy call: {e}')")

print("\n# Note: This is a conceptual Python call. Actual execution requires a working FSL installation and valid input files.")

### CLI Usage: `run_preprocessing correct_fsl`

The `eddy` correction can also be run using the `run_preprocessing.py` script. The subcommand is `correct_fsl`.

```bash
# Conceptual CLI command - replace paths and ensure files are correctly formatted
python cli/run_preprocessing.py correct_fsl \
  --dwi_file /path/to/your/dwi_to_correct.nii.gz \
  --bval_file /path/to/your/bvals.bval \
  --bvec_file /path/to/your/bvecs.bvec \
  --mask_file /path/to/your/brain_mask.nii.gz \
  --index_file /path/to/your/eddy_index.txt \
  --acqp_file /path/to/your/eddy_acqparams.txt \
  --out_base_name /path/to/your_output_dir/eddy_corrected_data \
  --repol True  # Example of passing an extra argument to eddy
  # Add --use_cuda if you have a GPU and eddy_cuda installed
  # Add --topup_fieldcoef /path/to/topup_fieldcoef.nii.gz and --topup_movpar /path/to/topup_movpar.txt if using topup results
```

## Note on `acqparams.txt` and `index.txt` Files

These files are critical for both `topup` and `eddy` to understand your data's acquisition parameters.

*   **`acqparams.txt` (or `encoding_file`)**: 
    *   Each line describes a unique acquisition scheme (combination of phase-encoding direction and readout time).
    *   Format per line: `PE_x  PE_y  PE_z  TotalReadoutTime`
        *   `PE_x, PE_y, PE_z`: Define the phase-encoding direction vector. For example:
            *   `0 1 0` could be Anterior-Posterior (AP).
            *   `0 -1 0` could be Posterior-Anterior (PA).
            *   `1 0 0` could be Right-Left (RL).
        *   `TotalReadoutTime`: The total readout time for the EPI sequence in seconds. This is often calculated as: `(EchoSpacing * (EPIFactor - 1))` if using Siemens terminology, or similar depending on scanner vendor.

*   **`index.txt`** (primarily for `eddy`):
    *   A text file with a single line of space-separated integers.
    *   The number of integers must match the number of volumes in your main DWI NIfTI file.
    *   Each integer is a 1-based index referring to a line in your `acqparams.txt` file.
    *   Example: If `acqparams.txt` has only one line describing all your DWI volumes, `index.txt` would contain `1 1 1 ... 1` (one '1' for each DWI volume).
    *   If you have combined data (e.g., some AP, some PA within the main DWI), `index.txt` would point to the respective lines in `acqparams.txt`.

## Conclusion

The FSL tools `topup` and `eddy` are industry standards for correcting common and complex distortions in dMRI data. The `diffusemri` library provides convenient Python wrappers to integrate these tools into your processing pipelines.

However, successfully using them requires:
1.  A working FSL installation with tools in your system PATH.
2.  Careful preparation of input NIfTI files (dMRI, b0s, masks).
3.  Accurate `acqparams.txt` and `index.txt` files that correctly describe your dMRI acquisition.

Always refer to the official FSL documentation for the most detailed and up-to-date information on these tools and their parameters. The examples in this notebook are intended to show *how* the `diffusemri` wrappers are called, rather than being a directly runnable end-to-end pipeline without user-specific data.